In [ ]:
import pandas as pd

# read bacteria_db metadata for merging with rafah genus
bacteria_db_metadata = pd.read_csv(str(snakemake.input.meta), sep='\t')
# split lineage column
bacteria_db_metadata[['superkingdom', 'phylum', 'class', 'order','family', 'genus', 'species']] = bacteria_db_metadata['Lineage'].str.split(';', expand=True)
bacteria_db_metadata['genus'] = bacteria_db_metadata['genus'].str.partition('__')[2]
bacteria_db_metadata2 = bacteria_db_metadata[['genus', 'Lineage']].drop_duplicates('genus')


# read host taxonomy results
spacer = pd.read_csv(str(snakemake.input.spacer))
blast = pd.read_csv(str(snakemake.input.blast))
phist = pd.read_csv(str(snakemake.input.phist))
votu = pd.read_csv(str(snakemake.input.votu))
rafah = pd.read_csv(str(snakemake.input.rafah), sep='\t')

sp_bl = spacer.merge(blast, on='viral_genome', how='outer')
sp_bl_ph = sp_bl.merge(phist, on='viral_genome', how='outer')
sp_bl_ph_votu = sp_bl_ph.merge(votu, on='viral_genome', how='outer')

rafah.rename(columns={'Predicted_Host':'genus', 'Variable':'viral_genome'}, inplace=True)
rafah_meta = rafah.merge(bacteria_db_metadata2, on='genus')
rafah_meta.rename(columns={'Lineage':'rafah_host_taxonomy'}, inplace=True)
bacteria_db_metadata2.to_csv('/labdata3/hoffdata/carsonjm/CarsonJM/phide_piper_test/test.csv')
rafah_final = rafah_meta[['viral_genome', 'rafah_host_taxonomy']]

sp_bl_ph_votu_ra = sp_bl_ph_votu.merge(rafah_final, on='viral_genome', how='outer')

sp_bl_ph_votu_ra.to_csv(str(snakemake.output), index=False)